## Transformers

In [213]:
! pip install transformers

In [214]:
import re
import pyarabic.araby as araby
from nltk.tokenize import word_tokenize
import joblib
import nltk
import re
import streamlit as st
import tweepy
import pandas as pd
import numpy as np
import torch
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForMaskedLM
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
from nltk.stem import ARLSTem
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
! pip install stop-words
from stop_words import get_stop_words


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [215]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment"

bert_tokenizer = AutoTokenizer.from_pretrained(model_id)
bert_model = AutoModelForMaskedLM.from_pretrained(model_id)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment were not used when initializing BertForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']

In [216]:
bert_tokenizer.vocab

{'التجربه': 27415,
 'يفقه': 28830,
 'حاجته': 24290,
 'قسط': 28113,
 'الباحث': 7576,
 'والذي': 4262,
 'والتن': 6987,
 'تبلغ': 9223,
 'الاتحادية': 17530,
 'مطلب': 14046,
 '##هيار': 28986,
 'غسالات': 15311,
 '##ساندة': 29709,
 'استهدف': 18269,
 'لإسرائيل': 22755,
 'فريد': 12049,
 'جعلك': 29416,
 'للف': 3554,
 'المدني': 4990,
 '##◣': 1672,
 'بإمكانك': 28549,
 'الطيب': 4820,
 'ونت': 6246,
 'تخص': 8572,
 '##մ': 1568,
 'السهر': 13778,
 'العقبة': 24051,
 'بكرا': 17073,
 'ظلام': 24073,
 'اللجنة': 4450,
 '##جيرات': 19760,
 'شكلك': 24909,
 '##فورد': 20383,
 '##اهره': 6838,
 'الأوضاع': 12148,
 'الأمريكى': 23459,
 'المصطفى': 23950,
 'تتم': 5346,
 '##ضافة': 3045,
 'جولات': 20827,
 '##ويب': 22410,
 'وخذ': 17903,
 'وحنان': 24935,
 'المناق': 17993,
 'نصف': 4771,
 'خطوات': 8473,
 'وغيرها': 5370,
 'القهوه': 16560,
 'وصاحب': 14019,
 'للحق': 27832,
 'الخزانات': 22807,
 'اعمل': 11352,
 '##һ': 1533,
 'الشبك': 9602,
 'الإحت': 14028,
 'رك': 4422,
 'أناقة': 27222,
 'بطاقه': 17362,
 'مث': 3066,
 'حائل': 10542,
 

In [217]:
!pip install simpletransformers

## Build Own classification Model

In [218]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro') , accuracy_score(labels,preds)


In [219]:
import pandas as pd
import numpy as np


In [220]:
dataset=pd.read_excel('/content/original_dataset.xlsx')

In [221]:
dataset=dataset[['review_description','rating']]
dataset.head()

,review_description,rating
0,سيئ جدا بعد الإصدار الجديد,-1
1,ابلكيشن زباله بجد,-1
2,سيئ التطبيق لايعمل,-1
3,للأسف التطبيق للأسوأ كان جدا رائع وسهل وبسيط ا...,-1
4,التحديث بطيئ جدا جدا عند الفتح,-1


In [222]:
dataset.rename(columns={'review_description':'text','rating': 'labels'}, inplace=True, errors='raise')

In [223]:
# Define emojis dictionary
emojis = {
    "🙂":"يبتسم",
    "😂":"يضحك",
    "💔":"قلب حزين",
    "🙂":"يبتسم",
    "❤️":"حب",
    "❤":"حب",
    "😍":"حب",
    "😭":"يبكي",
    "😢":"حزن",
    "😔":"حزن",
    "♥":"حب",
    "💜":"حب",
    "😅":"يضحك",
    "🙁":"حزين",
    "💕":"حب",
    "💙":"حب",
    "😞":"حزين",
    "😊":"سعادة",
    "👏":"يصفق",
    "👌":"احسنت",
    "😴":"ينام",
    "😀":"يضحك",
    "😌":"حزين",
    "🌹":"وردة",
    "🙈":"حب",
    "😄":"يضحك",
    "😐":"محايد",
    "✌":"منتصر",
    "✨":"نجمه",
    "🤔":"تفكير",
    "😏":"يستهزء",
    "😒":"يستهزء",
    "🙄":"ملل",
    "😕":"عصبية",
    "😃":"يضحك",
    "🌸":"وردة",
    "😓":"حزن",
    "💞":"حب",
    "💗":"حب",
    "😑":"منزعج",
    "💭":"تفكير",
    "😎":"ثقة",
    "💛":"حب",
    "😩":"حزين",
    "💪":"عضلات",
    "👍":"موافق",
    "🙏🏻":"رجاء طلب",
    "😳":"مصدوم",
    "👏🏼":"تصفيق",
    "🎶":"موسيقي",
    "🌚":"صمت",
    "💚":"حب",
    "🙏":"رجاء طلب",
    "💘":"حب",
    "🍃":"سلام",
    "☺":"يضحك",
    "🐸":"ضفدع",
    "😶":"مصدوم",
    "✌️":"مرح",
    "✋🏻":"توقف",
    "😉":"غمزة",
    "🌷":"حب",
    "🙃":"مبتسم",
    "😫":"حزين",
    "😨":"مصدوم",
    "🎼 ":"موسيقي",
    "🍁":"مرح",
    "🍂":"مرح",
    "💟":"حب",
    "😪":"حزن",
    "😆":"يضحك",
    "😣":"استياء",
    "☺️":"حب",
    "😱":"كارثة",
    "😁":"يضحك",
    "😖":"استياء",
    "🏃🏼":"يجري",
    "😡":"غضب",
    "🚶":"يسير",
    "🤕":"مرض",
    "‼️":"تعجب",
    "🕊":"طائر",
    "👌🏻":"احسنت",
    "❣":"حب",
    "🙊":"مصدوم",
    "💃":"سعادة مرح",
    "💃🏼":"سعادة مرح",
    "😜":"مرح",
    "👊":"ضربة",
    "😟":"استياء",
    "💖":"حب",
    "😥":"حزن",
    "🎻":"موسيقي",
    "✒":"يكتب",
    "🚶🏻":"يسير",
    "💎":"الماظ",
    "😷":"وباء مرض",
    "☝":"واحد",
    "🚬":"تدخين",
    "💐" : "ورد",
    "🌞" : "شمس",
    "👆" : "الاول",
    "⚠️" :"تحذير",
    "🤗" : "احتواء",
    "✖️": "غلط",
    "📍"  : "مكان",
    "👸" : "ملكه",
    "👑" : "تاج",
    "✔️" : "صح",
    "💌": "قلب",
    "😲" : "مندهش",
    "💦": "ماء",
    "🚫" : "خطا",
    "👏🏻" : "برافو",
    "🏊" :"يسبح",
    "👍🏻": "تمام",
    "⭕️" :"دائره كبيره",
    "🎷" : "ساكسفون",
    "👋": "تلويح باليد",
    "✌🏼": "علامه النصر",
    "🌝":"مبتسم",
    "➿"  : "عقده مزدوجه",
    "💪🏼" : "قوي",
    "📩":  "تواصل معي",
    "☕️": "قهوه",
    "😧" : "قلق و صدمة",
    "🗨": "رسالة",
    "❗️" :"تعجب",
    "🙆🏻": "اشاره موافقه",
    "👯" :"اخوات",
    "©" :  "رمز",
    "👵🏽" :"سيده عجوزه",
    "🐣": "كتكوت",
    "🙌": "تشجيع",
    "🙇": "شخص ينحني",
    "👐🏽":"ايدي مفتوحه",
    "👌🏽": "بالظبط",
    "⁉️" : "استنكار",
    "⚽️": "كوره",
    "🕶" :"حب",
    "🎈" :"بالون",
    "🎀":    "ورده",
    "💵":  "فلوس",
    "😋":  "جائع",
    "😛":  "يغيظ",
    "😠":  "غاضب",
    "✍🏻":  "يكتب",
    "🌾":  "ارز",
    "👣":  "اثر قدمين",
    "❌":"رفض",
    "🍟":"طعام",
    "👬":"صداقة",
    "🐰":"ارنب",
    "☂":"مطر",
    "⚜":"مملكة فرنسا",
    "🐑":"خروف",
    "🗣":"صوت مرتفع",
    "👌🏼":"احسنت",
    "☘":"مرح",
    "😮":"صدمة",
    "😦":"قلق",
    "⭕":"الحق",
    "✏️":"قلم",
    "ℹ":"معلومات",
    "🙍🏻":"رفض",
    "⚪️":"نضارة نقاء",
    "🐤":"حزن",
    "💫":"مرح",
    "💝":"حب",
    "🍔":"طعام",
    "❤︎":"حب",
    "✈️":"سفر",
    "🏃🏻‍♀️":"يسير",
    "🍳":"ذكر",
    "🎤":"مايك غناء",
    "🎾":"كره",
    "🐔":"دجاجة",
    "🙋":"سؤال",
    "📮":"بحر",
    "💉":"دواء",
    "🙏🏼":"رجاء طلب",
    "💂🏿 ":"حارس",
    "🎬":"سينما",
    "♦️":"مرح",
    "💡":"قكرة",
    "‼":"تعجب",
    "👼":"طفل",
    "🔑":"مفتاح",
    "♥️":"حب",
    "🕋":"كعبة",
    "🐓":"دجاجة",
    "💩":"معترض",
    "👽":"فضائي",
    "☔️":"مطر",
    "🍷":"عصير",
    "🌟":"نجمة",
    "☁️":"سحب",
    "👃":"معترض",
    "🌺":"مرح",
    "🔪":"سكينة",
    "♨":"سخونية",
    "👊🏼":"ضرب",
    "✏":"قلم",
    "🚶🏾‍♀️":"يسير",
    "👊":"ضربة",
    "◾️":"وقف",
    "😚":"حب",
    "🔸":"مرح",
    "👎🏻":"لا يعجبني",
    "👊🏽":"ضربة",
    "😙":"حب",
    "🎥":"تصوير",
    "👉":"جذب انتباه",
    "👏🏽":"يصفق",
    "💪🏻":"عضلات",
    "🏴":"اسود",
    "🔥":"حريق",
    "😬":"عدم الراحة",
    "👊🏿":"يضرب",
    "🌿":"ورقه شجره",
    "✋🏼":"كف ايد",
    "👐":"ايدي مفتوحه",
    "☠️":"وجه مرعب",
    "🎉":"يهنئ",
    "🔕" :"صامت",
    "😿":"وجه حزين",
    "☹️":"وجه يائس",
    "😘" :"حب",
    "😰" :"خوف و حزن",
    "🌼":"ورده",
    "💋":  "بوسه",
    "👇":"لاسفل",
    "❣️":"حب",
    "🎧":"سماعات",
    "📝":"يكتب",
    "😇":"دايخ",
    "😈":"رعب",
    "🏃":"يجري",
    "✌🏻":"علامه النصر",
    "🔫":"يضرب",
    "❗️":"تعجب",
    "👎":"غير موافق",
    "🔐":"قفل",
    "👈":"لليمين",
    "™":"رمز",
    "🚶🏽":"يتمشي",
    "😯":"متفاجأ",
    "✊":"يد مغلقه",
    "😻":"اعجاب",
    "🙉" :"قرد",
    "👧":"طفله صغيره",
    "🔴":"دائره حمراء",
    "💪🏽":"قوه",
    "💤":"ينام",
    "👀":"ينظر",
    "✍🏻":"يكتب",
    "❄️":"تلج",
    "💀":"رعب",
    "😤":"وجه عابس",
    "🖋":"قلم",
    "🎩":"كاب",
    "☕️":"قهوه",
    "😹":"ضحك",
    "💓":"حب",
    "☄️ ":"نار",
    "👻":"رعب",
    "❎":"خطء",
    "🤮":"حزن",
    '🏻':"احمر"
    }
def data_cleaning(x):
    if not isinstance(x, str):
        return x

    new = re.sub(r'[^ء-ي ]',' ',x)
    return new

def handle_emojis(text, emojis):
  if not isinstance(text, str):
    return text

  emoji_pattern = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
  return emoji_pattern.sub(lambda x: emojis.get(x.group(), ''), text)

def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

def remove_duplicates(text):
    words = text.split()
    unique_words = set(words)
    return ' '.join(unique_words)


In [224]:
dataset['text'] = dataset['text'].apply(handle_emojis, args=[emojis])
dataset['text']=dataset['text'].apply(data_cleaning)
dataset['text']=dataset['text'].apply(remove_diacritics)
dataset['text']=dataset['text'].apply(remove_duplicates)

In [225]:
dataset.head(10)

,text,labels
0,سيئ الإصدار بعد الجديد جدا,-1
1,بجد زباله ابلكيشن,-1
2,سيئ لايعمل التطبيق,-1
3,معاك بسهولة وسهل التطبيق الآن تجد معقد ولا رائ...,-1
4,الفتح التحديث جدا عند بطيئ,-1
5,الناس اغلب تطبيق اخر اي عليه مرررره معتمده بعد...,-1
6,شوفو معطله لكم دخول انا اعمل تسجيل يفتح حسابي ...,-1
7,تحدث ضغطت اذا التطبيق قمت على اللي لاهو ولاهو ...,0
8,ونفس جديد العملاء احذفه امس مافي اتواصلت يمكن ...,-1
9,صراحه شي موسف,-1


# **prepare dataset**

In [226]:
label_counts = dataset["labels"].value_counts()

# print the label counts
print(label_counts)


 0    23753
-1    23035
 1    20339
Name: labels, dtype: int64


In [227]:
def encode(x):
  if x==-1:
    return 0
  if x==0:
    return 1
  return 2

In [228]:
label_counts = dataset["labels"].value_counts()

# print the label counts
print(label_counts)

 0    23753
-1    23035
 1    20339
Name: labels, dtype: int64


In [229]:
dataset['labels']=dataset['labels'].apply(encode)

In [230]:
num_rows=dataset.shape[0]
dataset = dataset.sample(frac = 1)

In [231]:
train=dataset[:int(num_rows*1)]
test=dataset[int(num_rows*.97)+1:]


In [232]:
model_id = "CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment"

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=2, overwrite_output_dir=True)

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    model_id,
    num_labels=3,
    args=model_args,
    use_cuda=True
)

# **train model**

In [ ]:
# Train the model
model.train_model(train)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
2023-07-02 23:20:21.648 INFO    simpletransformers.classification.classification_utils:  Converting to features started. Cache is not used.


  0%|          | 0/67127 [00:00<?, ?it/s]

## **Evaluate the model**

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test, f1_multiclass=f1_multiclass)

print(result)

In [ ]:
result_train, model_outputs_train, wrong_predictions_train = model.eval_model(train, f1_multiclass=f1_multiclass)

print(result_train)

In [ ]:
pred=[]
pred_train=[]
for i in model_outputs:
  pred.append(np.argmax(i))
for i in model_outputs_train:
  pred_train.append(np.argmax(i))

In [ ]:
from sklearn.metrics import confusion_matrix,mean_squared_error,precision_score,recall_score,f1_score ,classification_report
from sklearn import metrics

In [ ]:
print("Test accuracy  :",metrics.accuracy_score(pred, test['labels']) *100 ,"%")
print("MSE [test]         :",mean_squared_error(test['labels'],pred ))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cf1 = confusion_matrix(test['labels'],pred)
sns.heatmap(cf1,annot=True,fmt = '.0f')
plt.xlabel('prediction')
plt.ylabel('Actual')
plt.title(' Confusion Matrix')
plt.show()
print("           classification_report for test")
print(classification_report(test['labels'], pred))
print("           classification_report for train")
print(classification_report(train['labels'],pred_train))
print("Train accuracy  :",metrics.accuracy_score(pred_train, train['labels']) *100 ,"%")

In [ ]:
dataset.head(20)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(dataset[50000:67000], f1_multiclass=f1_multiclass)



In [ ]:
wrong_predictions[:50]

In [ ]:
import joblib

# train a scikit-learn model
model = model

# save the model to a file
joblib.dump(model, 'New_NLP_Model_GP.pkl')